In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from datetime import datetime
from math import isnan

import threading
import time
import pandas
from pandas import DataFrame

In [2]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []  # Initialize variable to store candle

    def historicalData(self, reqId, bar):
        print(f'reqId: {reqId} Time: {bar.date} Close: {bar.close}')
        self.data.append([reqId, bar.date, bar.close])

In [3]:
app = IBapi()
app.connect('127.0.0.1', 7497, 10645)


def run_loop():
    app.run()

In [4]:
reqId_serial = 1

In [5]:
# Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(1)  # Sleep interval to allow time for connection to server

ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:cashhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [6]:
def create_contract(curr1, curr2):
    # Create contract object
    contract = Contract()
    contract.symbol = curr1
    contract.secType = 'CASH'
    contract.exchange = 'IDEALPRO'
    contract.currency = curr2
    return contract

In [7]:
EUR_USD_contract = create_contract('EUR', 'USD')

In [8]:
# Request historical candles
app.reqHistoricalData(reqId=1, contract=EUR_USD_contract, endDateTime='', durationStr='1 D', barSizeSetting='1 day',
                      whatToShow='MIDPOINT', useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])
time.sleep(2)  # sleep to allow enough time for data to be returned

reqId: 1 Time: 20220422 Close: 1.07925


In [9]:
df = pandas.DataFrame(app.data, columns=['reqId', 'DateTime', 'Close'])

In [10]:
df['DateTime'] = [datetime.strptime(dt, '%Y%m%d') for dt in df['DateTime']]

In [11]:
df

,reqId,DateTime,Close
0,1,2022-04-22,1.07925


In [12]:
def request_data(contract):
    global reqId_serial
    reqId_serial += 1
    app.reqHistoricalData(reqId=reqId_serial, contract=contract, endDateTime='', durationStr='1 D',
                          barSizeSetting='1 day',
                          whatToShow='MIDPOINT', useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])
    time.sleep(0.5)
    if app.data[len(app.data) - 1][0] != reqId_serial:
        return float('NaN')
    else:
        return app.data[len(app.data) - 1][2]

In [13]:
print(request_data(EUR_USD_contract))

reqId: 2 Time: 20220422 Close: 1.07935
1.07935


In [14]:
def fetch_exc_rate(base, quote):
    contract = create_contract(base, quote)
    return request_data(contract)

In [15]:
currencies = ['USD', 'EUR', 'GBP', 'JPY', 'AUD']

In [16]:
def fetch_all(curr_list):
    matrix = DataFrame(columns=curr_list, index=curr_list)
    for i in range(0, len(curr_list)):
        for j in range(0, len(curr_list)):
            if i == j:
                matrix[curr_list[i]][curr_list[j]] = 1
            else:
                matrix[curr_list[i]][curr_list[j]] = fetch_exc_rate(curr_list[i], curr_list[j])
    return matrix

In [41]:
exchange_rates = fetch_all(currencies)

ERROR 43 200 No security definition has been found for the request
ERROR 44 200 No security definition has been found for the request


reqId: 45 Time: 20220422 Close: 128.5795


ERROR 46 200 No security definition has been found for the request


reqId: 47 Time: 20220422 Close: 1.07875
reqId: 48 Time: 20220422 Close: 0.841
reqId: 49 Time: 20220422 Close: 138.7225
reqId: 50 Time: 20220422 Close: 1.4904
reqId: 51 Time: 20220422 Close: 1.2827


ERROR 52 200 No security definition has been found for the request


reqId: 53 Time: 20220422 Close: 164.9465
reqId: 54 Time: 20220422 Close: 1.77205


ERROR 55 200 No security definition has been found for the request
ERROR 56 200 No security definition has been found for the request
ERROR 57 200 No security definition has been found for the request
ERROR 58 200 No security definition has been found for the request


reqId: 59 Time: 20220422 Close: 0.72385


ERROR 60 200 No security definition has been found for the request
ERROR 61 200 No security definition has been found for the request


reqId: 62 Time: 20220422 Close: 93.076


In [56]:
def fill_in_nan(matrix):
    for col in matrix.columns:
        for row in matrix.index:
            if isnan(matrix[col][row]) & (not isnan(matrix[row][col])):
                matrix[col][row] = 1 / matrix[row][col]
    return matrix

In [59]:
def check_all_data(matrix):
    return not matrix.isnull().values.any()


In [57]:
exchange_rates = fill_in_nan(exchange_rates)

In [60]:
check_all_data(exchange_rates)

True

In [61]:
exchange_rates

,USD,EUR,GBP,JPY,AUD
USD,1,1.07875,1.2827,0.007777,0.72385
EUR,0.926999,1,1.189061,0.007209,0.670961
GBP,0.779606,0.841,1,0.006063,0.564318
JPY,128.5795,138.7225,164.9465,1,93.076
AUD,1.381502,1.4904,1.77205,0.010744,1


In [62]:
def check_arbitrage(result_dict, matrix, curr1, curr2, curr3):
    arbitrage_amount = matrix[curr1][curr2] * matrix[curr2][curr3] * matrix[curr3][curr1] - 1
    result_dict[curr1 + '.' + curr2 + '.' + curr3] = arbitrage_amount
    arbitrage_amount_rev = matrix[curr1][curr3] * matrix[curr3][curr2] * matrix[curr2][curr1] - 1
    result_dict[curr1 + '.' + curr3 + '.' + curr2] = arbitrage_amount_rev
    print(curr1 + '->' + curr2 + '->' + curr3 + '->' + curr1 + ': ' + "{:.6f}".format(arbitrage_amount))
    print(curr1 + '->' + curr3 + '->' + curr2 + '->' + curr1 + ': ' + "{:.6f}".format(arbitrage_amount_rev))
    return arbitrage_amount

In [63]:
def check_all_arbitrage(result_dict, matrix, currency_list):
    for i in range(0, len(currency_list)):
        for j in range(i + 1, len(currency_list)):
            for k in range(j + 1, len(currency_list)):
                check_arbitrage(result_dict, matrix, currency_list[i], currency_list[j], currency_list[k])
    return result_dict

In [65]:
results = dict()
results = check_all_arbitrage(results, exchange_rates, currencies)

USD->EUR->GBP->USD: 0.000001
USD->GBP->EUR->USD: -0.000001
USD->EUR->JPY->USD: 0.000125
USD->JPY->EUR->USD: -0.000125
USD->EUR->AUD->USD: 0.000070
USD->AUD->EUR->USD: -0.000070
USD->GBP->JPY->USD: 0.000107
USD->JPY->GBP->USD: -0.000107
USD->GBP->AUD->USD: -0.000001
USD->AUD->GBP->USD: 0.000001
USD->JPY->AUD->USD: -0.000040
USD->AUD->JPY->USD: 0.000040
EUR->GBP->JPY->EUR: -0.000018
EUR->JPY->GBP->EUR: 0.000018
EUR->GBP->AUD->EUR: -0.000071
EUR->AUD->GBP->EUR: 0.000071
EUR->JPY->AUD->EUR: 0.000015
EUR->AUD->JPY->EUR: -0.000015
GBP->JPY->AUD->GBP: 0.000068
GBP->AUD->JPY->GBP: -0.000068


In [66]:
results

{'USD.EUR.GBP': 6.488991888264906e-07,
 'USD.GBP.EUR': -6.488987677188973e-07,
 'USD.EUR.JPY': 0.00012518912815839833,
 'USD.JPY.EUR': -0.00012517345780238553,
 'USD.EUR.AUD': 7.0488991888773e-05,
 'USD.AUD.EUR': -7.048402354103978e-05,
 'USD.GBP.JPY': 0.0001065631758485619,
 'USD.JPY.GBP': -0.0001065518213479999,
 'USD.GBP.AUD': -1.2532158729650078e-06,
 'USD.AUD.GBP': 1.2532174431534315e-06,
 'USD.JPY.AUD': -4.006322789973371e-05,
 'USD.AUD.JPY': 4.006483302632091e-05,
 'EUR.GBP.JPY': -1.7974733731107584e-05,
 'EUR.JPY.GBP': 1.7975056827879143e-05,
 'EUR.GBP.AUD': -7.108829844348907e-05,
 'EUR.AUD.GBP': 7.109335234867231e-05,
 'EUR.JPY.AUD': 1.4630861574937981e-05,
 'EUR.AUD.JPY': -1.4630647515834205e-05,
 'GBP.JPY.AUD': 6.774897946093539e-05,
 'GBP.AUD.JPY': -6.774438984757936e-05}